In [3]:
# Insert code here.
!pip install flair
!pip install dill

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
from globalfn.full_text import get_text
from globalfn.annotations import annotation
from globalfn.alignments import all_alignments

import subprocess

In [ ]:
def word_alignments(src_ids, tgt_ids):
    """
    Inputs:
        - src_ids: sentence ID of the source sentence(s)
        - tgt_ids: sentence ID of the target sentence(s)
    
    Outputs:
        - line: a string of the aligned pair of sentences, separated by three pipes.
        - alignments_dict: a dictionary of mappings between the words in the source sentence 
        and the words in the target sentence
    """
    src_sent = ""
    tgt_sent = ""

    if type(src_ids) is int:
        src_sent += get_text(src_ids, tokenized=True)
    else:
        for src_id in src_ids:
            src_sent += get_text(src_id, tokenized=True) + ' '
        src_sent = src_sent.strip()
    
    for tgt_id in tgt_ids:
        tgt_sent += get_text(tgt_id, tokenized=True) + ' '
    tgt_sent = tgt_sent.strip()

    line = f"{src_sent} ||| {tgt_sent}"
    with open("wordalign/input.txt", "w+") as wf:
        wf.write(line)
    
    subprocess.run("/home/jovyan/work/fast_align/build/fast_align -i ~/work/wordalign/input.txt -d -o -v > ~/work/wordalign/forward.align", shell=True)
    subprocess.run("/home/jovyan/work/fast_align/build/fast_align -i ~/work/wordalign/input.txt -d -o -v -r > ~/work/wordalign/reverse.align", shell=True)
    subprocess.run("/home/jovyan/work/fast_align/build/atools -i ~/work/wordalign/forward.align -j ~/work/wordalign/reverse.align -c grow-diag-final-and > ~/work/wordalign/output.txt", shell=True)

    with open("wordalign/output.txt", 'r') as rf:
        result = rf.readline().strip().split(" ")
    alignments_dict = {int(src_word_idx): int(tgt_word_idx) for src_word_idx,tgt_word_idx in [list(tmp.split('-')) for tmp in result]}
    return line, alignments_dict

In [ ]:
src_lang = "en"
tgt_lang = 'pt'
count_exact = 0
count_total = 0
for src_ids, tgt_ids in all_alignments(f"{src_lang}-{tgt_lang}").items():
    if type(src_ids) is int and len(tgt_ids) == 1:
        if annotation(src_ids, verbose=False) or annotation(tgt_ids[0], verbose=False):
            # get word alignments
            line, alignments_dict = word_alignments(src_ids, tgt_ids)
            
            # map lu word index to its annotated frame
            src_luidx_to_frame = {}
            tgt_luidx_to_frame = {}
            for src_anno in annotation(src_ids):
                for idx, frame in enumerate(src_anno.tokenized_frame_idx):
                    if frame != "-":
                        src_luidx_to_frame[idx]=frame
            
            for tgt_anno in annotation(tgt_ids[0]):
                for idx, frame in enumerate(tgt_anno.tokenized_frame_idx):
                    if frame != "-":
                        tgt_luidx_to_frame[idx]=frame

            # align frame-evoking words and create the document
            for src_luidx in src_luidx_to_frame.keys():
                if src_luidx in alignments_dict and alignments_dict[src_luidx] in tgt_luidx_to_frame:
                    print(line, file=open(f"frame_projection_results/{src_lang}-{tgt_lang}.txt", "a+"))
                    print((src_luidx, src_luidx_to_frame[src_luidx]), '---', (alignments_dict[src_luidx], tgt_luidx_to_frame[alignments_dict[src_luidx]]), 
                          file=open(f"frame_projection_results/{src_lang}-{tgt_lang}.txt", "a+"))
                    if src_luidx_to_frame[src_luidx] == tgt_luidx_to_frame[alignments_dict[src_luidx]]:
                        count_exact += 1
                    count_total += 1


1020 (language en) is not annotated.
1048 (language en) is not annotated.
1049 (language en) is not annotated.
1097 (language en) is not annotated.
1103 (language en) is not annotated.
1105 (language en) is not annotated.
861 (language pt) is not annotated.
1138 (language en) is not annotated.
1153 (language en) is not annotated.
1161 (language en) is not annotated.
1221 (language en) is not annotated.
